# T2-IA 
### Feito por chamada de API pela chave gratuita do Gemini

In [3]:
%pip install -U google-generativeai

INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 5.2 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 5.5 MB/s  0:00:02m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 7.6 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 8.4 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23/23 [google-generativeai]ogle-generativeai]language]

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [33]:
%pip install python-dotenv


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [34]:
import csv
import os
from dotenv import load_dotenv
import google.generativeai as genai

load_dotenv()
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

#modelo utilizado
MODEL = "gemini-2.5-flash"


## PROMPT ZERO SHOT

In [65]:
def prompt_zero_shot(texto):
    return f"""
Tarefa: Identificar falácias lógicas em textos argumentativos.

Instruções:
Dado o texto abaixo, identifique quais as seguintes falácias estão presentes nele:
- Ad Hominem
- Ladeira Escorregadia
- Generalização Apressada
- Falso Dilema
- Apelo à Emoção
- Espantalho


IMPORTANTE:
- Use APENAS esses nomes de falácias, exatamente como escritos acima.
- Se não tiver certeza, use apenas as falácias que mais se encaixam.
- Produza a resposta em UMA ÚNICA LINHA, no seguinte formato:

  Falacias: <lista de falácias separadas por vírgula>; Explicacao: <explicação curta em português>

Exemplo de formato de saída:
  Falacias: Ad Hominem, Falso Dilema; Explicacao: texto curto explicando por que essas falácias aparecem.

Texto:
\"\"\"{texto}\"\"\"

Resposta:
"""

def main():
    model = genai.GenerativeModel(MODEL)
    ids_escolhidos = ["1", "2", "3", "5", "8", "12", "14", "17", "19", "20", "33", "37"]
    exemplos = []
    falacias_lista = [
    "Ad Hominem",
    "Ladeira Escorregadia",
    "Generalização Apressada",
    "Falso Dilema",
    "Apelo à Emoção",
    "Espantalho"
    ]

    with open("../dataset_falacias_logicas.csv", newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for coluna in reader: 
            exemplos.append({
                "id": coluna["id"],
                "texto": coluna["texto"]
            })
        exemplos = [c for c in exemplos if c["id"] in ids_escolhidos]
        shot_label = "shot1"
        nome_arquivo = f"zero_shot_respostas_{shot_label}.csv"
        caminho_arquivo = os.path.join("resultados", nome_arquivo)

        if not os.path.exists("resultados"):
            print("Pasta 'resultados/' não encontrada. Criando agora...")
            os.makedirs("resultados")

        if os.path.exists(caminho_arquivo): #verifica se o arquivo ja existe, se existir cria um com _novo, usamos isso pra testes com mesmo numero de shots pra ver o quanto variam as respostas
            print(f"Aviso: o arquivo {caminho_arquivo} já existe. Criando nova versão com final '_novo'.")
            base, ext = os.path.splitext(caminho_arquivo)
            caminho_arquivo = base + "_novo" + ext

    with open(caminho_arquivo, "w", newline="", encoding="utf-8") as f_out:
        writer = csv.DictWriter(f_out, fieldnames=["id", "texto","falacias_encontradas", "resposta"])
        writer.writeheader()

        for coluna in exemplos:
            texto = coluna["texto"]
            prompt = prompt_zero_shot(texto)
            response = model.generate_content(prompt)
            resposta_texto = " ".join(response.text.split()).strip() #pra deixar formatado no csv
            
            #pra fazer a coluna de falacias encontradas no csv
            falacias_encontradas = [f for f in falacias_lista if f.lower() in resposta_texto.lower()]
            falacias_str = ", ".join(falacias_encontradas)
            writer.writerow({
                "id": coluna["id"],
                "texto": texto,
                "falacias_encontradas": falacias_str,
                "resposta": resposta_texto
            })
            print("\n===============================")
            print(f"Feito teste zero-shot com texto ID: {coluna['id']}")
            print(f"Texto: {texto}")
            print(f"Falácias: {falacias_str if falacias_str else 'nenhuma detectada'}")
            print(f"Resposta: {resposta_texto}")
            print("===============================\n")

if __name__ == "__main__":
    main()


Feito teste zero-shot com texto ID: 1
Texto: Esse cara mentiu. Deixe eu falar pra vocês, esse cara mentiu sobre Ben Carson quando ele tirou os votos do Ben Carson em Iowa. E ele continua. Esse cara vai falar qualquer coisa. Cara nojento. Agora eu sei porque ele não tem patrocínios de nenhum dos seus colegas.
Falácias: Ad Hominem, Generalização Apressada
Resposta: Falacias: Ad Hominem, Generalização Apressada; Explicacao: O texto ataca o caráter do indivíduo com insultos ("Cara nojento") e críticas à sua reputação ("não tem patrocínios"), em vez de seus argumentos (Ad Hominem). Além disso, ele faz uma conclusão ampla de que "vai falar qualquer coisa" baseado em uma única alegação de mentira (Generalização Apressada).


Feito teste zero-shot com texto ID: 2
Texto: Pessoa A: Acho que precisamos discutir formas de reduzir o número de armas em circulação no Brasil. Pessoa B: Então você quer desarmar completamente a população e deixar os cidadãos indefesos contra criminosos?
Falácias: Espan

## PROMPT FEW-SHOT

In [66]:
def prompt_few_shot(texto, exemplos_fewshot):
    exemplos_formatados = ""

    # Exemplos few-shot já no formato desejado
    for ex in exemplos_fewshot:
        exemplos_formatados += (
            "Exemplo:\n"
            f"Texto: \"{ex['texto']}\"\n"
            "Saída esperada (formato obrigatório):\n"
            f"Falacias: {ex['falacia']}; "
            f"Explicacao: {ex['resposta']}\n\n"
        )

    # (Opcional) debug pra ver como ficou
    print("\n===== EXEMPLOS FEW-SHOT FORMATADOS =====")
    print(exemplos_formatados)
    print("========================================\n")

    return f"""
Tarefa: Identificar falácias lógicas em textos argumentativos.

A seguir estão alguns exemplos de como identificar falácias lógicas em textos,
bem como o formato EXATO de saída que deve ser usado.

{exemplos_formatados}
Agora analise o próximo texto.

Instruções gerais:
Dado o texto abaixo, identifique quais das seguintes falácias estão presentes nele:
- Ad Hominem
- Ladeira Escorregadia
- Generalização Apressada
- Falso Dilema
- Apelo à Emoção
- Espantalho

IMPORTANTE:
- Use APENAS esses nomes de falácias, exatamente como escritos acima.
- Se não tiver certeza, use apenas as falácias que mais se encaixam.
- Produza a resposta em UMA ÚNICA LINHA, no seguinte formato:

Agora, analise o seguinte texto:

Texto:
\"\"\"{texto}\"\"\"

Resposta:
"""

def main():
    model = genai.GenerativeModel(MODEL)

    ids_escolhidos = ["1", "2", "3", "5", "8", "12", "14", "17", "19", "20", "33", "37"]
    exemplos = []
    ids_exemplos_fewshot = ["9", "11", "5", "6", "23", "22"] #exemplos que irao ser usados no prompt few-shot
    exemplos_fewshot = []

    falacias_lista = [
        "Ad Hominem",
        "Ladeira Escorregadia",
        "Generalização Apressada",
        "Falso Dilema",
        "Apelo à Emoção",
        "Espantalho"
    ]

    #le csv
    with open("../dataset_falacias_logicas.csv", newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for coluna in reader:
            if coluna["id"] in ids_exemplos_fewshot:
                exemplos_fewshot.append({
                    "id": coluna["id"],
                    "texto": coluna["texto"],
                    "falacia": coluna["falacias_esperadas"], 
                    "resposta": coluna["resposta_esperada"]
                })

            exemplos.append({
                "id": coluna["id"],
                "texto": coluna["texto"]    
            })

        exemplos = [c for c in exemplos if c["id"] in ids_escolhidos]

        shot_label = "shot1"
        nome_arquivo = f"few_shot_respostas_{shot_label}.csv"
        caminho_arquivo = os.path.join("resultados", nome_arquivo)

        if not os.path.exists("resultados"):
            print("Pasta 'resultados/' não encontrada. Criando agora...")
            os.makedirs("resultados")

        if os.path.exists(caminho_arquivo):
            print(f"Aviso: o arquivo {caminho_arquivo} já existe. Criando nova versão com final '_novo'.")
            base, ext = os.path.splitext(caminho_arquivo)
            caminho_arquivo = base + "_novo" + ext

    with open(caminho_arquivo, "w", newline="", encoding="utf-8") as f_out:
        writer = csv.DictWriter(f_out, fieldnames=["id", "texto", "falacias_encontradas", "resposta"])
        writer.writeheader()

        for coluna in exemplos:
            texto = coluna["texto"]
            prompt = prompt_few_shot(texto, exemplos_fewshot)
            response = model.generate_content(prompt)

            resposta_texto = " ".join(response.text.split()).strip()

            falacias_encontradas = [f for f in falacias_lista if f.lower() in resposta_texto.lower()]
            falacias_str = ", ".join(falacias_encontradas)

            writer.writerow({
                "id": coluna["id"],
                "texto": texto,
                "falacias_encontradas": falacias_str,
                "resposta": resposta_texto
            })
            print("\n===============================")
            print(f"Feito teste few-shot com texto ID: {coluna['id']}")
            print(f"Texto: {texto}")
            print(f"Falácias: {falacias_str if falacias_str else 'nenhuma detectada'}")
            print(f"Resposta: {resposta_texto}")
            print("===============================\n")

            
if __name__ == "__main__":
    main()


===== EXEMPLOS FEW-SHOT FORMATADOS =====
Exemplo:
Texto: "Fui pra França no Natal, minha experiência lá com os franceses foi horrível. Até fui roubada quando visitei o museu do Louvre. Quando for pra Europa, saiba que a França é linda, mas os franceses são arrogantes e ladrões."
Saída esperada (formato obrigatório):
Falacias: Generalização Apressada; Explicacao: Generaliza a partir de poucos casos.

Exemplo:
Texto: "Ou você apoia a reforma agraria, ou está contra o progresso e contra os direitos dos trabalhadores."
Saída esperada (formato obrigatório):
Falacias: Falso Dilema; Explicacao: Apresenta apenas duas opções extremas.

Exemplo:
Texto: "As pessoas que criticam o uso de remédios naturais só querem vender mais remédios caros e não se preocupam com a saúde de verdade."
Saída esperada (formato obrigatório):
Falacias: Ad Hominem; Explicacao: Ataca motivação em vez de argumentar.

Exemplo:
Texto: "Se aceitarmos que os esportes se tornem mais inclusivos para pessoas transgênero, logo 

## PROMPT ZERO-SHOT-CHAIN-OF-THOUGHT

In [67]:
def prompt_zero_shot_CoT(texto):
    return f"""
Tarefa: Identificar falácias lógicas em textos argumentativos.

Instruções:
Dado o texto abaixo, identifique quais as seguintes falácias estão presentes nele:
- Ad Hominem
- Ladeira Escorregadia
- Generalização Apressada
- Falso Dilema
- Apelo à Emoção
- Espantalho

Indique em uma linha de texto de no máximo 120 palavras a resposta que deve conter:
- As falácias (nomes).
- Indique em qual/quais trechos do texto argumentativo as falácias aparecem.
- Dê uma breve explicação do por que aquele trecho representa essa falácia.

Texto:
\"\"\"{texto}\"\"\"

Resposta: 
(Pense passo a passo para identificar, classificar e explicar as falácias presentes no texto acima.)
(Mostre APENAS a resposta final, curta e organizada. Não revele o raciocínio.)

"""

def main():
    model = genai.GenerativeModel(MODEL)
    ids_escolhidos = ["1", "2", "3", "5", "8", "12", "14", "17", "19", "20", "33", "37"]
    exemplos = []
    falacias_lista = [
    "Ad Hominem",
    "Ladeira Escorregadia",
    "Generalização Apressada",
    "Falso Dilema",
    "Apelo à Emoção",
    "Espantalho"
    ]

    with open("../dataset_falacias_logicas.csv", newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for coluna in reader: 
            exemplos.append({
                "id": coluna["id"],
                "texto": coluna["texto"]
            })
        exemplos = [c for c in exemplos if c["id"] in ids_escolhidos]
        shot_label = "shot1"
        nome_arquivo = f"zero_shot_cot_respostas_{shot_label}.csv"
        caminho_arquivo = os.path.join("resultados", nome_arquivo)

        if not os.path.exists("resultados"):
            print("Pasta 'resultados/' não encontrada. Criando agora...")
            os.makedirs("resultados")

        if os.path.exists(caminho_arquivo): #verifica se o arquivo ja existe, se existir cria um com _novo, usamos isso pra testes com mesmo numero de shots pra ver o quanto variam as respostas
            print(f"Aviso: o arquivo {caminho_arquivo} já existe. Criando nova versão com final '_novo'.")
            base, ext = os.path.splitext(caminho_arquivo)
            caminho_arquivo = base + "_novo" + ext

    with open(caminho_arquivo, "w", newline="", encoding="utf-8") as f_out:
        writer = csv.DictWriter(f_out, fieldnames=["id", "texto","falacias_encontradas", "resposta"])
        writer.writeheader()

        for coluna in exemplos:
            texto = coluna["texto"]
            prompt = prompt_zero_shot(texto)
            response = model.generate_content(prompt)
            resposta_texto = " ".join(response.text.split()).strip() #pra deixar formatado no csv
            
            #pra fazer a coluna de falacias encontradas no csv
            falacias_encontradas = [f for f in falacias_lista if f.lower() in resposta_texto.lower()]
            falacias_str = ", ".join(falacias_encontradas)
            writer.writerow({
                "id": coluna["id"],
                "texto": texto,
                "falacias_encontradas": falacias_str,
                "resposta": resposta_texto
            })
            print("\n===============================")
            print(f"Feito teste zero-shot com texto ID: {coluna['id']}")
            print(f"Texto: {texto}")
            print(f"Falácias: {falacias_str if falacias_str else 'nenhuma detectada'}")
            print(f"Resposta: {resposta_texto}")
            print("===============================\n")

if __name__ == "__main__":
    main()


Feito teste zero-shot com texto ID: 1
Texto: Esse cara mentiu. Deixe eu falar pra vocês, esse cara mentiu sobre Ben Carson quando ele tirou os votos do Ben Carson em Iowa. E ele continua. Esse cara vai falar qualquer coisa. Cara nojento. Agora eu sei porque ele não tem patrocínios de nenhum dos seus colegas.
Falácias: Ad Hominem, Generalização Apressada
Resposta: Falacias: Ad Hominem, Generalização Apressada; Explicacao: O texto ataca o caráter da pessoa ("Cara nojento", "Esse cara mentiu") em vez de refutar seus argumentos (Ad Hominem) e faz uma generalização ampla sobre seu comportamento futuro ("Esse cara vai falar qualquer coisa") baseada em um número limitado de supostas ações passadas (Generalização Apressada).


Feito teste zero-shot com texto ID: 2
Texto: Pessoa A: Acho que precisamos discutir formas de reduzir o número de armas em circulação no Brasil. Pessoa B: Então você quer desarmar completamente a população e deixar os cidadãos indefesos contra criminosos?
Falácias: Lade

In [ ]:
import csv
import os

FALACIA_COLS = [
    "ad_hominem",
    "ladeira_escorregadia",
    "gen_apressada",
    "falso_dilema",
    "apelo_emocao",
    "espantalho",
]

# mapeia pedaços de texto (em minúsculas) para a coluna correspondente
KEYWORD_TO_COL = {
    "ad hominem": "ad_hominem",
    "ladeira escorregadia": "ladeira_escorregadia",
    "declive escorregadio": "ladeira_escorregadia",  # sinônimo
    "generalização apressada": "gen_apressada",
    "generalizacao apressada": "gen_apressada",
    "falso dilema": "falso_dilema",
    "apelo à emoção": "apelo_emocao",
    "apelo a emoção": "apelo_emocao",
    "apelo a emocao": "apelo_emocao",
    "espantalho": "espantalho",
}

def extrai_bins(falacias_str: str) -> dict:
    """Recebe string tipo 'Ad Hominem, Generalização Apressada'
       e devolve dict {col: 0/1}."""
    row_bins = {col: 0 for col in FALACIA_COLS}
    if not falacias_str:
        return row_bins

    s = falacias_str.lower()

    for key, col in KEYWORD_TO_COL.items():
        if key in s:
            row_bins[col] = 1

    return row_bins


def converte_resultado_para_binario(caminho_in, caminho_out=None):
    """Lê um csv de respostas (com falacias_encontradas)
       e gera outro com colunas binárias."""
    if caminho_out is None:
        base, ext = os.path.splitext(caminho_in)
        caminho_out = base + "_binario" + ext

    with open(caminho_in, newline="", encoding="utf-8") as f_in, \
         open(caminho_out, "w", newline="", encoding="utf-8") as f_out:

        reader = csv.DictReader(f_in)
        fieldnames = ["id", "texto"] + FALACIA_COLS
        writer = csv.DictWriter(f_out, fieldnames=fieldnames)
        writer.writeheader()

        # opcional: debug pra ver colunas do csv de entrada
        print("Colunas de entrada:", reader.fieldnames)

        for row in reader:
            falacias_str = row.get("falacias_encontradas", "")
            bins = extrai_bins(falacias_str)

            out_row = {
                "id": row["id"],
                "texto": row["texto"],
            }
            out_row.update(bins)
            writer.writerow(out_row)

    print(f"Arquivo binário criado em: {caminho_out}")
    return caminho_out


In [77]:
import csv

ids_escolhidos = ["1","2","3","5","8","12","14","17","19","20","33","37"]

def gerar_gabarito_binario(
    ids_escolhidos,
    caminho_dataset="../dataset_falacias_logicas.csv",
    pasta_saida="resultados",
    nome_base="gabarito_binario_shot1"
):
    if not os.path.exists(pasta_saida):
        os.makedirs(pasta_saida)

    caminho_saida = os.path.join(pasta_saida, f"{nome_base}.csv")

    fieldnames = [
        "id", "texto",
        "ad_hominem", "ladeira_escorregadia", "gen_apressada",
        "falso_dilema", "apelo_emocao", "espantalho"
    ]

    with open(caminho_dataset, newline="", encoding="utf-8") as f_in:
        reader = csv.DictReader(f_in)
        with open(caminho_saida, "w", newline="", encoding="utf-8") as f_out:
            writer = csv.DictWriter(f_out, fieldnames=fieldnames)
            writer.writeheader()
            for row in reader:
                if row["id"] in ids_escolhidos:
                    writer.writerow({
                        "id": row["id"],
                        "texto": row["texto"],
                        "ad_hominem": row["ad_hominem"],
                        "ladeira_escorregadia": row["ladeira_escorregadia"],
                        "gen_apressada": row["gen_apressada"],
                        "falso_dilema": row["falso_dilema"],
                        "apelo_emocao": row["apelo_emocao"],
                        "espantalho": row["espantalho"],
                    })

    print(f"Gabarito binário criado em: {caminho_saida}")
    return caminho_saida

gerar_gabarito_binario(ids_escolhidos)

Gabarito binário criado em: resultados/gabarito_binario_shot1.csv


'resultados/gabarito_binario_shot1.csv'

# Analise dos resultados obtidos

In [70]:
# colunas binárias (iguais às do dataset_falacias_logicas.csv)
FALACIA_COLS = [
    "ad_hominem",
    "ladeira_escorregadia",
    "gen_apressada",
    "falso_dilema",
    "apelo_emocao",
    "espantalho",
]

def carrega_gabarito_binario(caminho_dataset):
    """
    Lê o gabarito binário e devolve:
    gabarito[id] = {coluna_falacia: 0 ou 1}
    """
    gabarito = {}
    with open(caminho_dataset, newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            gabarito[row["id"]] = {col: int(row[col]) for col in FALACIA_COLS}
    return gabarito


def avalia_binario(caminho_resultado_bin, gabarito_bin):
    precisions, recalls, f1s, exacts = [], [], [], []

    with open(caminho_resultado_bin, newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for col in reader:
            id_ = col["id"]
            if id_ not in gabarito_bin:
                continue  

            y_true_dict = gabarito_bin[id_]
            true_set = {c for c, v in y_true_dict.items() if v == 1}

            # previsto (do csv binário de respostas)
            y_pred_dict = {c: int(col[c]) for c in FALACIA_COLS}
            pred_set = {c for c, v in y_pred_dict.items() if v == 1}

            tp = len(true_set & pred_set)
            fp = len(pred_set - true_set)
            fn = len(true_set - pred_set)

            precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
            recall    = tp / (tp + fn) if (tp + fn) > 0 else 0.0
            f1        = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0
            exact     = 1.0 if true_set == pred_set else 0.0

            precisions.append(precision)
            recalls.append(recall)
            f1s.append(f1)
            exacts.append(exact)

    n = len(precisions) or 1
    print(f"\n=== Avaliação: {caminho_resultado_bin} ===")
    print(f"Precision médio: {sum(precisions)/n:.2f}")
    print(f"Recall médio:    {sum(recalls)/n:.2f}")
    print(f"F1 médio:        {sum(f1s)/n:.2f}")
    print(f"Exact match:     {sum(exacts)/n:.2f}")

In [78]:
gabarito_bin = carrega_gabarito_binario(
    "resultados/gabarito_binario_shot1.csv"
)

converte_resultado_para_binario(
    "resultados/zero_shot_respostas_shot1.csv"
)
converte_resultado_para_binario(
    "resultados/few_shot_respostas_shot1.csv"
)
converte_resultado_para_binario(
    "resultados/zero_shot_cot_respostas_shot1.csv"
)

avalia_binario(
    "resultados/zero_shot_respostas_shot1_binario.csv",
    gabarito_bin
)

avalia_binario(
    "resultados/few_shot_respostas_shot1_binario.csv",
    gabarito_bin
)

avalia_binario(
    "resultados/zero_shot_cot_respostas_shot1_binario.csv",
    gabarito_bin
)

Colunas de entrada: ['id', 'texto', 'falacias_encontradas', 'resposta']
Arquivo binário criado em: resultados/zero_shot_respostas_shot1_binario.csv
Colunas de entrada: ['id', 'texto', 'falacias_encontradas', 'resposta']
Arquivo binário criado em: resultados/few_shot_respostas_shot1_binario.csv
Colunas de entrada: ['id', 'texto', 'falacias_encontradas', 'resposta']
Arquivo binário criado em: resultados/zero_shot_cot_respostas_shot1_binario.csv

=== Avaliação: resultados/zero_shot_respostas_shot1_binario.csv ===
Precision médio: 0.61
Recall médio:    0.92
F1 médio:        0.72
Exact match:     0.17

=== Avaliação: resultados/few_shot_respostas_shot1_binario.csv ===
Precision médio: 0.82
Recall médio:    0.88
F1 médio:        0.84
Exact match:     0.67

=== Avaliação: resultados/zero_shot_cot_respostas_shot1_binario.csv ===
Precision médio: 0.63
Recall médio:    0.92
F1 médio:        0.73
Exact match:     0.25
